In [127]:
import requests
import json
import pickle
import time

In [121]:
def save_pickle(data, path):
    f = open(path, 'wb')
    pickle.dump(data, f)
    f.close()
    
def load_pickle(path):
    f1 = open(path, 'rb')
    data = pickle.load(f1)
    f1.close()
    return data
save_pickle('aaaa', 'data/a.pkl')
load_pickle('data/a.pkl')

'aaaa'

In [141]:
def process_bar(percent, total_length=10, start_str='|', end_str='100%'):
    bar = ''.join(["\033[31m%s\033[0m"%'###'] * int(percent * total_length)) + ''
    bar = '\r' + start_str + bar.ljust(total_length) + ' {:0>4.1f}%|'.format(percent*100) + end_str
    print(bar, end='', flush=True)
 
 
for i in range(101):
    time.sleep(0.1)
    process_bar(i/100)

|############################## 100.0%|100%

In [21]:
headers = {
    "Host": "m.weibo.cn",
    "Referer": "https://m.weibo.cn/p/230413",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36",
    "X-Requested-with": "XMLHttpRequest"
}
data = 'ner'

In [108]:
# 获取微博页数
uid = "6569231942"
url = "https://m.weibo.cn/profile/info?uid={0}".format(uid)
headers['Referer'] = "https://m.weibo.cn/profile/{0}".format(uid)
try:
    response = requests.get(url, headers)
    if response.status_code==200:
        content = response.json()
        page_count = content['data']['user']['statuses_count'] // 10 + 1
except requests.ConnectionError as e:
    print("error",e.args)
page_count

5

In [122]:
def getPageJson(uid, page):
    
    url = "https://m.weibo.cn/api/container/getIndex?containerid=230413{0}&".format(uid)
    headers['Referer'] = headers['Referer'] + uid
    
    """
    获取单个页面的json数据
    :param page:传入的page参数
    :return:返回页面响应的json数据
    """
    url = url + "page=%d"%page
    try:
        response = requests.get(url, headers)
        if response.status_code==200:
            return response.json()
    except requests.ConnectionError as e:
        print("error",e.args)
page_idx = 5
page_json = getPageJson('6569231942', page_idx)  # 散0207

/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,


In [123]:
'''
    解析json，获取图片、视频的url
'''
data_list = []
items = page_json.get("data").get("cards")
for item in items[:]:
    if 'card_group' in item:
        for card in item.get('card_group'):
            pics = card.get("mblog").get("pics")
            # 有些微博没有配图，所以需要加一个判断，方便后面遍历不会出错
            if pics is not None:
                for pic in pics:
                    pic_dict = {}
                    pic_dict["pid"] = pic.get("pid")
                    pic_dict["url"] = pic.get("large").get("url")
                    data_list.append(pic_dict)
                    print(pic_dict)
    else:
        mblog = item.get('mblog') 
        pics = mblog.get("retweeted_status").get("pics")
        # 有些微博没有配图，所以需要加一个判断，方便后面遍历不会出错
        if pics is not None:
            for pic in pics:
                pic_dict = {}
                pic_dict["pid"] = pic.get("pid")
                pic_dict["url"] = pic.get("large").get("url")
                data_list.append(pic_dict)
                print(pic_dict)
        else: # 转发的视频
            page_info = mblog.get("retweeted_status").get("page_info")
            urls = page_info.get('urls')
            video_dict = {}
            if 'object_id' not in page_info:
                print(mblog["retweeted_status"]['text'].split('<')[0], page_info['page_url'])
                continue
            video_dict["vid"] = page_info.get("object_id")
            video_dict["url"] = urls['mp4_720p_mp4']
            data_list.append(video_dict)
            print(video_dict)

{'pid': '006AfEgvgy1gkx3bnei62j31pn2nje87', 'url': 'https://wx1.sinaimg.cn/large/006AfEgvgy1gkx3bnei62j31pn2nje87.jpg'}
{'pid': '006AfEgvgy1gkx3bt3wapj31qr2qv7wo', 'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3bt3wapj31qr2qv7wo.jpg'}
{'pid': '006AfEgvgy1gkx398qxnsj31i22e9b2d', 'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx398qxnsj31i22e9b2d.jpg'}
{'pid': '006AfEgvgy1gkx3bykn4ij31rk2tpe87', 'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx3bykn4ij31rk2tpe87.jpg'}
{'pid': '006AfEgvgy1gkx3cobtpgj31t32vdqvc', 'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3cobtpgj31t32vdqvc.jpg'}
{'pid': '006AfEgvgy1gkx3cvxt70j320k340x6v', 'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx3cvxt70j320k340x6v.jpg'}
{'pid': '006AfEgvgy1gkx3d38955j31yr340x6w', 'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3d38955j31yr340x6w.jpg'}
{'pid': '006AfEgvgy1gkx3d9q4b9j3200340e88', 'url': 'https://wx1.sinaimg.cn/large/006AfEgvgy1gkx3d9q4b9j3200340e88.jpg'}
{'pid': '006AfEgvgy1gkx3dgdi62j31r52tp4q

In [124]:
data_list

[{'pid': '006AfEgvgy1gkx3bnei62j31pn2nje87',
  'url': 'https://wx1.sinaimg.cn/large/006AfEgvgy1gkx3bnei62j31pn2nje87.jpg'},
 {'pid': '006AfEgvgy1gkx3bt3wapj31qr2qv7wo',
  'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3bt3wapj31qr2qv7wo.jpg'},
 {'pid': '006AfEgvgy1gkx398qxnsj31i22e9b2d',
  'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx398qxnsj31i22e9b2d.jpg'},
 {'pid': '006AfEgvgy1gkx3bykn4ij31rk2tpe87',
  'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx3bykn4ij31rk2tpe87.jpg'},
 {'pid': '006AfEgvgy1gkx3cobtpgj31t32vdqvc',
  'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3cobtpgj31t32vdqvc.jpg'},
 {'pid': '006AfEgvgy1gkx3cvxt70j320k340x6v',
  'url': 'https://wx2.sinaimg.cn/large/006AfEgvgy1gkx3cvxt70j320k340x6v.jpg'},
 {'pid': '006AfEgvgy1gkx3d38955j31yr340x6w',
  'url': 'https://wx3.sinaimg.cn/large/006AfEgvgy1gkx3d38955j31yr340x6w.jpg'},
 {'pid': '006AfEgvgy1gkx3d9q4b9j3200340e88',
  'url': 'https://wx1.sinaimg.cn/large/006AfEgvgy1gkx3d9q4b9j3200340e88.jpg'},
 {'pid':

In [140]:
def save_img_video(results):
    """
    下载图片
    :param results:
    :return:
    """
    count = len(results)
    path = 'data/'
    idx = 0
    for r_dict in results:
        if 'pid' in r_dict:
            name = r_dict.get("pid") + ".jpg"
        elif 'vid' in r_dict:
            name = r_dict.get("vid") + ".mp4"
        else:
            print("未知错误: " + r_dict)
            continue
        try: 
            data = requests.get(r_dict.get("url"))
            with open(path + name, "wb") as file:
                file.write(data.content)
                file.close()
                idx += 1
        except Exception as e:
            print(name+"\tdownload failed!", r_dict)
            continue
#         process_bar(i/(count-1))
    if count == idx:
         print("all file download successed!")
save_img_video(data_list)

|################################################################## 227.3%|100%

KeyboardInterrupt: 

In [149]:
count = len(data_list)
idx = 0
for r_dict in data_list:
    time.sleep(0.1)
    process_bar(idx/(count-1))
    idx += 1

|############################## 100.0%|100%

In [83]:
mp4_720p_mp4 = 'https://f.video.weibocdn.com/ET2YgH1Olx07NgWRPZcY0104120060cs0E010.mp4?label=mp4_720p&template=720x1280.24.0&trans_finger=c3f00996be5378650057cf237d7bfffd&ori=0&ps=1CwnkDw1GXwCQx&Expires=1635566474&ssig=vmgofrVxAM&KID=unistore,video'
path = 'data/'
img_data = requests.get(mp4_720p_mp4)
with open(path + 'test.mp4', "wb") as file:
    file.write(img_data.content)
    file.close()

/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
